In [ ]:
%load_ext autoreload
%autoreload 2

from process_examples import load_examples, extract_terms_from_example
from utils import load_conceptnet, normalize_conceptnet, normalize_input
from find_shortest_path import find_word_path, render_path_verbose, search_shortest_path

# Step 1: Implement the path search

## Preprocessing of the knowledge base

I am using ConceptNet for this assignment. In order to work effectively with it, I downloaded the
knowledgebase (`conceptnet-assertions-5.7.0.csv.gz`) and transformed it. I discarded all non-English
nodes (i.e. all nodes that don't start with `/c/en`). 

Afterwards I normalized all node labels by removing the prefix (`/c/en`) and possible suffixes (e.g.
`/n`, `/v`), replacing `_` with a space, applying lowercase and applying the `WordNetLemmatizer`
from `nltk`. Most of this normalization is not strictly needed at this point, as labels in
ConceptNet are already normalized. However, it is still done at this point to ensure that node
labels and input terms are normalized in exactly the same way. The normalization allows us to fuzzy
match terms to ConceptNet nodes (e.g. because mouse and mice is normalized to the same base form),
while still being able to find a matching node for a term in $O(1)$ (on average) using Python
dictionaries. "Real" fuzzy matching like substring matching would require $O(n)$ for matching nodes
and terms which is considerably slower.

TODO fix duplicate entries issue

After preprocessing, the knowledge base is represented by this six data structures:

- `nodes_idx2name: list[str]` mapping node indices to normalized node names
- `nodes_name2idx: dict[str, int]` mapping node names to indices
- `labels_idx2name: list[str]` mapping edge label indices to labels
- `labels_name2idx: dict[str, int]` mapping edge labels to indices
- `adjacency_lists: dict[int, set[int]]` list of adjacent nodes for each node
- `edge_descriptors: dict[tuple[int, int], set[EdgeDescriptor]]` maps a pair of indices to all
  direct edges in ConceptNet between this two nodes. An edge descriptor contains the edge label
  index, weight and row index in the CSV-file (for lookup of further attributes, if necessary)

In order to enable the path search, to find reversed edges, the adjacency lists are undirected.
`edge_descriptors` is still directed. By combining information from both data structures, the path
visualization function can find out if an edge was traversed in original or reversed direction by
the path finding algorithm.

## Path Finding

I implemented a standard breadth-first search for finding paths in the knowledge graphs. I augmented the
algorithm to keep track of the distance between the currently traversed node and the start node.
This information is used to stop the search after exceeding the maximum path length. I decided to
ignore edge weights to keep the path finding simple, especially since ConceptNet often contains
multiple edges between two nodes with different weights.

TODO add code

# Step 2: Use the search function and visualize the paths

I used Spacy for extracting terms. I extract all tokens and noun phrases, that are not in Spacy's
stopword list. This method allows to find all relevant terms with a high probability, even though it
also extracts many irrelevant terms (high recall, low precision). After evaluating which extracted
terms are not present in ConceptNet, I augmented the code to remove articles (the, a, an) from noun
phrases. Spacy always adds articles to a noun phrase if present, while ConceptNet stores nodes
labels without article. This significantly reduced the number of out-of-vocabulary terms.

TODO reduced by how much

## Maximum Path Length

I experimented with different maximum paths lengths, but I found that a path length of four may
already need a considerable amount of time while not providing very useful connections. 

Examples
- `checked --HasContext--> north america --HasContext--> canada <--HasContext-- garbage` (10 sec)
- `safe --RelatedTo--> heavy <--RelatedTo-- carry --Antonym--> baggage` (50 sec)
- `excited <--HasSubevent-- score home run --HasPrerequisite--> play baseball --HasPrerequisite--> apply` (10 sec)
- `harvard --IsA--> college --HasContext--> canada <--HasContext-- letter` (4 sec)

Finding a path with path lengths three on the other hand usually takes less than one second and
often provides useful connections (see the following examples). Therefore I decided to use a maximum
path length of three.

## Example Visualizations

Looking for paths between all terms in the question and context on the one side and the answer
choices on the other side is a huge number of potential paths (for example 1, there are 63
combinations and for 39 of them a path was found). Because of this, I will only show an excerpt of
all generated paths here (You can find all of them in the appendix).

